In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
%matplotlib inline

In [2]:
#db settings
host = 'database-airbnb.cl4bob5uux9z.us-west-1.rds.amazonaws.com'
dbname= 'postgres'
user ='postgres'
port = 5432
password ='geslozapostgre123'

# PYTHON FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, dbname
        )
    )
  
  
if __name__ == '__main__':
  
    try:
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(
            f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

Connection to the database-airbnb.cl4bob5uux9z.us-west-1.rds.amazonaws.com for user postgres created successfully.


In [77]:
query = "SELECT host_id, host_name, room_type, COUNT (DISTINCT minimum_nights) as min_nights, \
COUNT (DISTINCT id) as nr_listings, MAX (accommodates) as accommodates, \
CAST(AVG(DISTINCT price) as decimal(10)) as price, SUM (number_of_reviews_ltm) AS number_of_reviews_year, \
CAST(AVG(avg_score_rating) as DECIMAL(10,2)) as avg_rating, \
CAST(AVG (review_scores_rating) as DECIMAL(10,2)) AS scores_avg, CAST(AVG (review_scores_cleanliness) as DECIMAL(10,2)) AS clean_avg, \
CAST(AVG (review_scores_checkin) as DECIMAL(10,2)) AS checkin_avg, CAST(AVG (review_scores_communication) as DECIMAL(10,2)) AS comm_avg, \
CAST(AVG (review_scores_location) as DECIMAL(10,2)) AS location_avg, CAST(AVG (review_scores_value) as DECIMAL(10,2)) AS val_avg \
FROM listings WHERE number_of_reviews_ltm > 0 AND \
host_is_superhost = 0 AND neighbourhood = 'Wieden'\
GROUP BY host_id, host_name, room_type \
HAVING COUNT (DISTINCT id) = 1"
potential_listings_wieden = pd.read_sql_query(query, con=engine)
potential_listings_wieden.sort_values(by=['number_of_reviews_year'], ascending=False).head(20)

,host_id,host_name,room_type,min_nights,nr_listings,accommodates,price,number_of_reviews_year,avg_rating,scores_avg,clean_avg,checkin_avg,comm_avg,location_avg,val_avg
34,149786100.0,Christoph,Entire home/apt,1,1,2,45.0,44.0,4.72,4.55,4.56,4.86,4.79,4.96,4.53
1,833967.0,Annemarie And Motti,Entire home/apt,1,1,2,31.0,42.0,4.80,4.68,4.77,4.94,4.94,4.78,4.74
54,335869217.0,Anna,Entire home/apt,1,1,2,59.0,41.0,4.57,4.37,4.20,4.73,4.76,4.88,4.37
39,164523634.0,Mario,Entire home/apt,1,1,3,67.0,38.0,4.70,4.69,4.55,4.87,4.55,4.85,4.65
25,52428878.0,Ogi,Entire home/apt,1,1,2,45.0,37.0,4.66,4.57,4.28,4.74,4.82,4.83,4.67
53,318961878.0,Mel,Entire home/apt,1,1,2,43.0,31.0,4.45,4.22,3.84,4.75,4.81,4.66,4.34
51,300103129.0,Rudolf,Entire home/apt,1,1,2,61.0,30.0,4.56,4.36,4.17,4.82,4.77,4.83,4.42
29,70495983.0,Han,Entire home/apt,1,1,5,83.0,29.0,4.76,4.63,4.57,4.93,4.90,4.87,4.61
20,35275274.0,Boris,Entire home/apt,1,1,2,44.0,25.0,4.90,4.88,4.82,4.98,4.97,4.90,4.81
33,116849625.0,Diana,Entire home/apt,1,1,4,62.0,22.0,4.59,4.30,4.33,4.94,4.76,4.94,4.43


In [4]:
listings = pd.read_sql_query("SELECT * FROM listings WHERE neighbourhood = 'Wieden' and room_type = 'Entire home/apt'", con=engine)
listings.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,...,accommodates,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,avg_score_rating
0,242141.0,Vienna terrace-apartment on U1/Central Station,1310252.0,Maria,Wieden,48.18746,16.37478,Entire home/apt,54.0,1.0,...,5,4.61,4.80,4.53,4.88,4.91,4.85,4.64,1,4.75
1,272589.0,Cozy Studio Apt - directly in the heart of Vie...,1426583.0,Cecilia & Walfried,Wieden,48.19916,16.36657,Entire home/apt,139.0,14.0,...,3,5.00,4.86,5.00,5.00,5.00,5.00,4.71,0,4.94
2,275667.0,NEWLY RENOVATED DOWNTOWN FLAT #7,1440944.0,Stephanie,Wieden,48.19204,16.36592,Entire home/apt,42.0,2.0,...,2,4.78,4.88,4.81,4.77,4.51,4.86,4.71,1,4.76
3,275668.0,UR OWN DOWNTOWN VIENNA FLAT #15,1440944.0,Stephanie,Wieden,48.19328,16.36755,Entire home/apt,50.0,2.0,...,4,4.70,4.83,4.75,4.80,4.50,4.86,4.76,1,4.74
4,289195.0,Apartment Kalrskirche 1,685572.0,Christoph,Wieden,48.19762,16.37036,Entire home/apt,216.0,3.0,...,7,4.93,4.93,4.90,4.93,4.90,4.93,4.76,1,4.90


In [ ]:
15883.0
listings_test = pd.read_sql_query("SELECT * FROM listings WHERE host_id =", con=engine)
listings_test.head()

In [72]:
# query for host, rating distribution
listings_single = listings.query("host_id == 116849625.0")
listings_single

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,...,accommodates,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,avg_score_rating
262,39970114.0,Beautiful Apartment in the middle of the Center,116849625.0,Diana,Wieden,48.19581,16.37289,Entire home/apt,62.0,1.0,...,4,4.3,4.46,4.33,4.94,4.76,4.94,4.43,1,4.59


In [73]:
#find room type of host to compare
listings_types = listings.query("neighbourhood == 'Wieden' & room_type == 'Entire home/apt' & host_is_superhost == 0 & price <= 75 & avg_score_rating <= 4.55")

In [74]:
#superhost charge rate
listings_types_superhost = listings.query("neighbourhood == 'Wieden' & room_type == 'Entire home/apt' & host_is_superhost == 1 & price <= 90")

In [75]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="avg", y=listings_single['price'], x=listings_single['room_type'], name="Average host price", texttemplate="%{y:.2}"))
fig.add_trace(go.Histogram(histfunc="avg", y=listings_types['price'], x=listings_single['room_type'], name=" Average price of similar listings", texttemplate="%{y:.2}"))
fig.show()

In [71]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="avg", y=listings_single['price'], x=listings_single['room_type'], name="Average host price", texttemplate="%{y:.2}"))
fig.add_trace(go.Histogram(histfunc="avg", y=listings_types_superhost['price'], x=listings_types_superhost['room_type'], name=" Average price of super host listings", texttemplate="%{y:.2}"))
fig.show()

In [66]:
average_score_all = listings['avg_score_rating']
average_score_single = listings_single['avg_score_rating']
average_clean_all = listings['review_scores_cleanliness']
average_clean_single = listings_single['review_scores_cleanliness']
average_checkin_all = listings['review_scores_checkin']
average_checkin_single = listings_single['review_scores_checkin']
average_comm_all = listings['review_scores_communication']
average_comm_single = listings_single['review_scores_communication']
average_value_all = listings['review_scores_value']
average_value_single = listings_single['review_scores_value']
fig = go.Figure()

fig.add_trace(go.Bar(
    x= ['average rating'],
    y=average_score_all,
    legendgroup='group',
    name="Competition"
))

fig.add_trace(go.Bar(
    x= ['average rating'],
    y=average_score_single,
    legendgroup='group',
    name="Host"
))

fig.add_trace(go.Bar(
    x= ['cleanliness rating'],
    y=average_clean_all,
    legendgroup='group2',
    name="Competition"
))

fig.add_trace(go.Bar(
    x= ['cleanliness rating'],
    y=average_clean_single,
    legendgroup='group2',
    name="Host"
))

fig.add_trace(go.Bar(
    x= ['checkin rating'],
    y=average_checkin_all,
    legendgroup='group3',
    name="Competition"
))

fig.add_trace(go.Bar(
    x= ['checkin rating'],
    y=average_checkin_single,
    legendgroup='group3',
    name="Host"
))

fig.add_trace(go.Bar(
    x= ['communication rating'],
    y=average_comm_all,
    legendgroup='group4',
    name="Competition"
))

fig.add_trace(go.Bar(
    x= ['communication rating'],
    y=average_comm_single,
    legendgroup='group4',
    name="Host"
))

fig.add_trace(go.Bar(
    x= ['value rating'],
    y=average_value_all,
    legendgroup='group5',
    name="Competition"
))

fig.add_trace(go.Bar(
    x= ['value rating'],
    y=average_value_single,
    legendgroup='group5',
    name="Host"
))
fig.update_layout(barmode='group')
fig.update_traces(texttemplate='%{y:.3}', textposition='outside')
fig.show()